# Databricks
Multi-cloud Lakehouse Platform base on Apache Spark. A Lakehouse is a unified analytics platform which combines the best elements of a data warehouse and data lake. It is one platform that unifies data engineering, analytics and AI workloads.

## Data Lake
* Open, flexible with ML support

## Data warehouse
* Reliable, performative, with strong governance.

# Databricks Lakehouse Platform.
* Clouse Service - Runs on AWS, Azure and Google Cloud
* Runtime - Spark and Delta Lake
* Workspace - Data Engineering, Data Warehousing and Machine Learning

## Control Plane (Databricks Account)
* Web UI, Cluster Management, Workflows and Notebooks.

## Data Plane (Customer Account)
* Cluster VMs and Storage (DBFS)

## Spark on Databricks
* In-memory, distributed data processing.
* Supports Scala, Python, SQL, R and Java
* Support batch and stream processing
* Support structured, semi and unstructured data

## DBFS (Databricks File System)
* Distributed File System
* Comes pre-installed on the cluster
* Abstraction layer (data is persisted in underlying cloud storage)

## Cluster
A set of computers of nodes working together like a single entity.
* Master Node (Driver)
  * Coordinates the workers and their parallel execution of tasks.
* Worker Nodes



# Delta Lake
Delta Lake is an open-source storage framework which brings reliability to data lakes.

Is|Is Not
-|-
Open-source | Proprietary Technology
Storage framework/layer | Storage format/medium
Enabling building Lakehouse | Data warehouse / Database service

## Transaction log (Delta log)
* The _delta_log is stored in JSON format
  * 000.json, 001.json, 002.json
  * Ordered records of every transaction performed on the table
  * JSON file contains commit information
  * Operations performed and predicates used
  * Data files affected (added/removed)
* The data files are store in parquet format

The transaction log is the single source of truth for your data.


## Delta Lake Advantages
* Brings ACID transactions to object storage
* Provides scalable metadata
* Full audit trail of all changes


# Magic Commands
* %fs
* %md
* %sql
* %python

# TABLES
## CREATE TABLE
```sql
CREATE TABLE employees
--USING DELTA
(id INT, name STRING, salary DOUBLE)
```
* Requires a manual schema declaration
* Needs an INSERT to insert data.
* Using Delta is optional as all tables are delta tables by default on Databricks.
* Tables are created and can be viewed in the Data tab using the Catalog explorer.

## CREATE TABLE AS
* Data is inserted automatically from source table.
* Schema information is automatically inferred.
```sql
CREATE TABLE
COMMENT "Enter a comment"
PARTITIONED BY (col1, col2)
LOCATION '/custom/path/'
AS
```
## IF NOT EXISTS
```sql
CREATE TABLE IF NOT EXISTS table-name
```
## Table Constraint
* Two types
 * NOT NULL
 * CHECK
```sql
ALTER TABLE
```
## Deep Clone
* Copies data and metadata.
```sql
CREATE TABLE table-name
DEEP CLONE source-table
```
* Can sync changes between the source and target table. So changes can happen incrementally.
* Large datasets can take a long time.
* Data modifications will not affect source table.
## Shallow Clone
* Copies only metadata. No data is moved.
* Good for testing changes to new table without modifying the original table.
* Data modifications will not affect source table.
```sql
CREATE TABLE table-name
SHALLOW CLONE source-table
```
## INSERT INTO
Records are inserted in parallel depending on the number of cores you are using. If you have 4 cores and 4 records to INSERT then you should see 4 files.



# Views
* Queries the source tables defined in the view, but does not have any data of it's own.
* Types
  * Stored Views
    * Persisted in the database
    * CREATE VIEW view-name AS query
  * Temporary View
    * Dropped when the session ends.
      * Spark session is create when opening a notebook, attaching a re-attaching a notebook, installing a python package, restarting cluster.
    * CREATE TEMP VIEW view-name as query
  * Global Temporary View
    * Tied to the cluster
      * Persists until dropped or cluster stops.
    * CREATE GLOBAL TEMP VIEW view-name AS
      * SELECT * FROM global_temp view-name

#Metadata
## DESCRIBE DETAILS
Let's us look at table Metadata.
* Use DESCRIBE EXTENDED to see location information or if table is managed.
* Use DESCRIBE DATABASE EXTENDED to see database information.

## DESCRIBE HISTORY
Shows all operations performed on the table.

## SHOW
* SHOW TABLE IN database-name
* SHOW TABLE IN global_temp;

# Time Travel
Audit data changes
Use DESCRIBE HISTORY to look at changes.

* Use a timestamp
TIMESTAMP AS OF "timestamp"
* Use a version
VERSION AS OF <number>
<table>@v<number>

## RESTORE TABLE


# OPTIMIZE
* Compacts small files into larger files to improve performance.
* ZORDER BY column_name
  * Sorts data and notes the begin and end in each file (doesn't have to look through all files).

# Vacuum Command
* Cleans up uncommited files and files that are not in the latest table.
* VACUUM table_name [retention period]
  * Default retention is 7 days.

**Once you run VACUUM you can no longer time travel past the retention period as the files no longer exist!**

Get past the rentention period by using:
VACUUM table_name RETAIN 0 HOURS
SET spark.databricks.delta.retentionDurationCheck = false;

Command will work now, but do not do it in production.



# Relational Entities
## Databases
* Schema in Hive metastore. Can use CREATE DATABASE or CREATE SCHEMA.
  * The Hive metastore stores metadata about yoru data warehouse.
  * Every workspace has a default database in the Hive metastore.
  * Default database location: dbfs:/user/hive/warehouse
  * Default database
    * dbfs:/user/hive/warehouse/table-1
  * Other databases created
    * dbfs:/user/hive/warehouse/custom-db-name.db/table-1
### Custom Database
* dbfs:/custom/path/to/db/custom-db-name.db/table-1

## Tables
* Two types of Tables
  * Managed Tables
    * Table created under default database.
    * Databricks manages so when you delete a table the underlying data is removed.
  * External Tables
    * Table created outside the database directory.
    * Use CREATE TABLE LOCATION 'path'
    * Databricks does not manage so when you delete a table the underlying data is NOT removed.
## Location Keyword
  * This is where the data files are located.
  * You can have external tables in the default database.
  * USE database; will let you create your table in any database you want to.
```sql
  USE DATABASE
  CREATE TABLE
  LOCATION 'path'
```

# Querying Files
* SELECT
```sql
SELECT *
FROM file_format.`/file/path/file-name`
```
  * SELECT is good for self-describing formats (formats that contain metadata) like json and parquet.
  * Not good for csv, tsv, customer delimiter.

* Wildcards
  * Use wildcards to query multiple files
    * file_*.txt
* Raw data extract
  *file_format
    * Text
      * JSON, CSV, TSV, TXT
        * Use text as file_format
        * Useful if corrupted
    * Binary
      * Use binaryFile as file_format
* CREATE TABLE from a file
  * USE CTAS statement
  * Useful for data with defined schemas.
  * Does not support any file options.
* CREATE TABLE table_name USING
```sql
CREATE TABLE table_name
(schema)
USING data_source
OPTIONS(key1=val1,...)
LOCATION = 'path'
```
  * USING can be JDBC, CSV and others.
  * Always creates an external table
  * The table is **NON-DELTA** table so it is not a managed table.
      * **Solution is to create a TEMP VIEW and then use CTAS statement on TEMP VIEW to create the table.***

    
  